Testing new visualisation features
---

---

In [1]:
from pathlib import Path
import os
_cwd = os.getcwd()

In [2]:
os.chdir(Path(_cwd)/'..') # .\SAE\
os.getcwd()

'C:\\Users\\Giulia\\Study\\projects\\research\\SAE'

In [3]:
import numpy as np
import torch
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
from models import GenerativeAE
import itertools

In [4]:
from experiments import EvaluationManager

In [17]:
import importlib
importlib.reload(EvaluationManager)

<module 'metrics' from 'C:\\Users\\Giulia\\Study\\projects\\research\\SAE\\metrics\\__init__.py'>

In [23]:
from metrics import utils 
importlib.reload(utils)

<module 'metrics.utils' from 'C:\\Users\\Giulia\\Study\\projects\\research\\SAE\\metrics\\utils.py'>

In [24]:
from experiments.EvaluationManager import ModelHandler

In [25]:
handler = ModelHandler(model_name="BaseSAE", model_version="dummy", data="MNIST")

Found existing config file: loading.
BaseSAE model loaded.


In [26]:
checkpoints = handler.list_available_checkpoints()

Available checkpoints at logs\BaseSAE\dummy_MNIST\checkpoints :
['logs\\BaseSAE\\dummy_MNIST\\checkpoints\\epoch=0-step=87.ckpt', 'logs\\BaseSAE\\dummy_MNIST\\checkpoints\\epoch=1-step=89.ckpt', 'logs\\BaseSAE\\dummy_MNIST\\checkpoints\\epoch=2-step=91.ckpt', 'logs\\BaseSAE\\dummy_MNIST\\checkpoints\\epoch=3-step=93.ckpt', 'logs\\BaseSAE\\dummy_MNIST\\checkpoints\\epoch=4-step=95.ckpt', 'logs\\BaseSAE\\dummy_MNIST\\checkpoints\\epoch=5-step=99.ckpt']


In [27]:
handler.load_checkpoint()

Loading latest checkpoint at logs\BaseSAE\dummy_MNIST\checkpoints\epoch=5-step=99.ckpt .


In [28]:
scores = handler.score_model(FID=False, disentanglement=True)

Scoring model disentanglement.
DCI scoring


AssertionError: 